In [3]:
import cv2 as cv
import numpy as np

### Reading and displaying the constituent images

In [4]:
image1 = cv.imread("/home/gousepeer/Desktop/SEM6/VR/Assignment2/images/Image1.jpeg")
cv.imshow("Image 1",image1)

cv.waitKey(1)

-1

In [5]:
image2 = cv.imread("/home/gousepeer/Desktop/SEM6/VR/Assignment2/images/Image2.jpeg")
cv.imshow("Image 2",image2)

cv.waitKey(1)

-1

### Class to create a panorama

In [6]:
class makePanorama:
    def __init__(self,image1,image2):
        self.image1 = image1
        self.image2 = image2
        self.method = "orb"


    def resizeImage(self,image,dimensions):
       
        resizedImage = cv.resize(image, dimensions)
        return resizedImage


    def grayScaleImage(self,image):
        
        grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        return grayImage


    def preprocessImages(self,dimensions):
       
        self.image1 = self.resizeImage(self.image1,dimensions)
        self.image2 = self.resizeImage(self.image2,dimensions)

        self.grayImage1 = self.grayScaleImage(self.image1)
        self.grayImage2 = self.grayScaleImage(self.image2)


    def createDescriptor(self,nfeatures=2000):
       
        if self.method == 'sift':
            descriptor = cv.xfeatures2d.SIFT_create(nfeatures=nfeatures)
        elif self.method == 'surf':
            descriptor = cv.xfeatures2d.SURF_create(nfeatures=nfeatures)
        elif self.method == 'brisk':
            descriptor = cv.BRISK_create()
        elif self.method == 'orb':
            descriptor = cv.ORB_create(nfeatures=2000)

        self.descriptor = descriptor
        return self.descriptor


    def getKeypointDescriptors(self,image):
       
        self.createDescriptor()
        (keypoints, descriptors) = self.descriptor.detectAndCompute(image, None)        
        return (keypoints, descriptors)  


    def drawKeypoints(self,image,keypoints,desc):
       
        cv.imshow(desc,cv.drawKeypoints(image, keypoints, None, (245, 228, 52)))
        cv.waitKey(1)      


    def createMatcher(self,crossCheck):        
       
        if self.method == 'sift' or self.method == 'surf':
            self.bf = cv.BFMatcher(cv.NORM_L2, crossCheck=crossCheck)
        elif self.method == 'orb' or self.method == 'brisk':
            self.bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=crossCheck)
        return self.bf


    def ratioTest(self,rawMatches,ratio):
      
        matches = []
        for m,n in rawMatches:
            if m.distance < n.distance * ratio:
                matches.append(m)
        return matches        

    
    def matchPointsKNN(self,img1Features,img2Features,ratio):
      
        self.createMatcher(False)
        rawMatches = self.bf.knnMatch(img1Features,img2Features,2)
        matches = self.ratioTest(rawMatches,ratio)
        return matches


    def drawMatchingStiches(self,keypointsImage1,keypointsImage2):
       
        img3 = cv.drawMatches(self.image1,keypointsImage1,self.image2,keypointsImage2,np.random.choice(matches,100), None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        cv.imshow("image3",img3)
        cv.waitKey(1)

    
    def stitchImages(self,img1, img2, H):
       
        rows1, cols1 = img1.shape[:2]
        rows2, cols2 = img2.shape[:2]

        PointsListImage1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
        fovPoints = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)

        PointsListImage2 = cv.perspectiveTransform(fovPoints, H)
        PointsList = np.concatenate((PointsListImage1,PointsListImage2), axis=0)

        [xmin, ymin] = np.int32(PointsList.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(PointsList.max(axis=0).ravel() + 0.5)
        
        transformDistance = [-xmin,-ymin]
        
        HTranslation = np.array([[1, 0, transformDistance[0]], [0, 1, transformDistance[1]], [0, 0, 1]])

        outputImage = cv.warpPerspective(img2, HTranslation.dot(H), (xmax - xmin, ymax - ymin))
        outputImage[transformDistance[1]:rows1 + transformDistance[1], transformDistance[0]:cols1 + transformDistance[0]] = img1

        return outputImage


    def getHomographyMatrix(self,matchCount,keypointsImage1,keypointsImage2):
      
        if len(matches) > matchCount:
            Image1Points = np.float32([ keypointsImage1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
            Image2Points = np.float32([ keypointsImage2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)

            matrix,mask = cv.findHomography(Image1Points, Image2Points, cv.RANSAC,5.0)
            self.result = self.stitchImages(self.image2,self.image1, matrix)
            return matrix,mask,self.result 


    def getImage1(self):
     
        return self.image1

    def getImage2(self):
        
        return self.image2

    def setMethod(self,method):
        
        self.method = method

### Pre Processing the images

In [7]:
panorama = makePanorama(image1,image2)
panorama.preprocessImages((600,500))
panorama.setMethod("orb")

In [8]:
image1 = panorama.getImage1()
image2 = panorama.getImage2()

In [9]:
cv.imshow("Preprocessed Image 1",image1)
cv.waitKey(1)

-1

In [10]:
cv.imshow("Preprocessed Image 2",image2)
cv.waitKey(1)

-1

### Detect Distinctive Keypoints

In [11]:
(keypointsImage1,descriptorsImage1) = panorama.getKeypointDescriptors(image1)

In [12]:
(keypointsImage2,descriptorsImage2) = panorama.getKeypointDescriptors(image2)

In [13]:
panorama.drawKeypoints(panorama.image1,keypointsImage1,"Keypoints Image 1")

In [14]:
panorama.drawKeypoints(panorama.image2,keypointsImage2,"Keypoints Image 2")

### Draw Matcher Stiches

In [15]:
matches = panorama.matchPointsKNN(descriptorsImage1, descriptorsImage2, ratio=0.75)

In [16]:
panorama.drawMatchingStiches(keypointsImage1,keypointsImage2)

### Create Panorama

In [17]:
matrix,mask,result = panorama.getHomographyMatrix(10,keypointsImage1,keypointsImage2)

In [18]:
cv.imshow("result",result)
cv.waitKey(1)

cv.imwrite("result.jpg",result)

True

: 